In [1]:
import os

In [2]:
abs_path = '/home/Akshath/PycharmProjects/Summarize4Me'

os.chdir(abs_path)
%pwd

'/home/Akshath/PycharmProjects/Summarize4Me'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_path: Path

In [4]:
from Summarize4Me.constants import *
from Summarize4Me.utils.common import read_yaml, create_dirs

In [5]:
class ConfigManager:
    def __init__(
        self,
        config_fpath = CONFIG_FILE_PATH,
        params_fpath = PARAMS_FILE_PATH
    ):

        self.config = read_yaml(config_fpath)
        self.params = read_yaml(params_fpath)

        create_dirs([self.config.artifacts_root])

    def data_trans_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_dirs([config.root_dir])

        data_trans_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_path=config.tokenizer_path
        )

        return data_trans_config

In [6]:
import os
from Summarize4Me.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [7]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_path)

    def convert_to_features(self, batch):
        input_encodings=self.tokenizer(batch['dialogue'], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings=self.tokenizer(batch['summary'], max_length=240, truncation=True)

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': input_encodings['input_ids']
        }

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))


In [8]:
try:
    config = ConfigManager()
    data_transformation_config = config.data_trans_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise

[2023-06-06 10:09:08,513: INFO: common: YAML File: config/config.yaml loaded]
[2023-06-06 10:09:08,515: INFO: common: YAML File: params.yaml loaded]
[2023-06-06 10:09:08,516: INFO: common: Created dir at: artifacts]
[2023-06-06 10:09:08,519: INFO: common: Created dir at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/home/Akshath/PycharmProjects/Summarize4Me/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]